# Optimise the performance of an AI Agent


Optimise the performance of an AI Agent

This lab exercise is designed to help participants understand and evaluate AI agents through a series of structured tasks, using IBM's Granite
model from Replicate for enhanced reasoning capabilities.

## Task 1:

### Setup and Initialization
- Setting up credentials
- Loading the Granite model via Replicate
- Basic environment configuration

In [1]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-k01y0ais
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-k01y0ais
  Resolved https://github.com/ibm-granite-community/utils to commit c9a6b769ec5f436629cecf649afcd8f130908c30
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.6 MB

In [2]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.7/733.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.4 MB/s eta 0:00:00


In [ ]:
import os
import replicate
import json
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

print('✅ Setup complete!')

## Task 2:
###Create Simple Tool Calling & Tools
- Define Custom Tools
- Build a functional agent that can use custom tools to perform specific tasks

In [ ]:
from typing import Dict, List, Optional
import json

class SimpleAgent:
    def __init__(self):
        """Initialize the SimpleAgent with product database"""
        self.products = [
            {
                "id": "P010",
                "name": "Wireless Headphones",
                "category": "Electronics",
                "description": "High-quality wireless headphones with noise cancellation",
                "price": 149.99,
                "stock": 75,
                "delivery": {
                    "shipping_cost": 4.99,
                    "free_shipping_threshold": 100,
                    "estimated_days": "2-3"
                }
            },
            {
                "id": "P022",
                "name": "Running Shoes",
                "category": "Sports",
                "description": "Lightweight running shoes with cushioned soles",
                "price": 89.99,
                "stock": 120,
                "delivery": {
                    "shipping_cost": 5.99,
                    "free_shipping_threshold": 100,
                    "estimated_days": "3-5"
                }
            },
            {
                "id": "P035",
                "name": "Coffee Maker",
                "category": "Kitchen",
                "description": "Programmable coffee maker with 12-cup capacity",
                "price": 79.99,
                "stock": 45,
                "delivery": {
                    "shipping_cost": 7.99,
                    "free_shipping_threshold": 100,
                    "estimated_days": "2-4"
                }
            },
            {
                "id": "P012",
                "name": "Laptop Stand",
                "category": "Electronics",
                "description": "Adjustable aluminum laptop stand with cooling",
                "price": 29.99,
                "stock": 150,
                "delivery": {
                    "shipping_cost": 3.99,
                    "free_shipping_threshold": 50,
                    "estimated_days": "1-2"
                }
            },
            {
                "id": "P007",
                "name": "Yoga Mat",
                "category": "Sports",
                "description": "Non-slip yoga mat with carrying strap",
                "price": 24.99,
                "stock": 95,
                "delivery": {
                    "shipping_cost": 4.99,
                    "free_shipping_threshold": 50,
                    "estimated_days": "2-3"
                }
            },
            {
                "id": "P045",
                "name": "Wireless Mouse",
                "category": "Electronics",
                "description": "Ergonomic Wireless Mouse with Rechargable Battery",
                "price": 49.99,
                "stock": 18,
                "delivery": {
                    "shipping_cost": 4.99,
                    "free_shipping_threshold": 50,
                    "estimated_days": "2-3"
                }
            }
        ]

    def get_product_info(self, product_id: str) -> str:
        """Tool 1: Get product description and category"""
        product = self._find_product(product_id)
        if not product:
            return "Product not found."

        return f"Product: {product['name']}\nCategory: {product['category']}\nDescription: {product['description']}"

    def get_price(self, product_id: str) -> str:
        """Tool 2: Get product price information"""
        product = self._find_product(product_id)
        if not product:
            return "Product not found."

        return f"Price: ${product['price']:.2f}"

    def get_delivery_info(self, product_id: str) -> str:
        """Tool 3: Get delivery details"""
        product = self._find_product(product_id)
        if not product:
            return "Product not found."

        delivery = product['delivery']
        free_shipping_msg = f"\nFree shipping on orders over ${delivery['free_shipping_threshold']:.2f}"

        return (f"Shipping Cost: ${delivery['shipping_cost']:.2f}\n"
                f"Estimated Delivery: {delivery['estimated_days']} business days"
                f"{free_shipping_msg}")

    def check_stock(self, product_id: str) -> str:
        """Tool 4: Check stock availability"""
        product = self._find_product(product_id)
        if not product:
            return "Product not found."

        stock = product['stock']
        if stock > 100:
            status = "High Stock"
        elif stock > 50:
            status = "Good Stock"
        elif stock > 10:
            status = "Limited Stock"
        else:
            status = "Low Stock"

        return f"Stock Status: {status} ({stock} units available)"

    def _find_product(self, product_id: str) -> Optional[Dict]:
        """Helper method to find a product by ID"""
        for product in self.products:
            if product['id'] == product_id:
                return product
        return None

## 3. Evaluation Function

Let's create a simple function to evaluate our agent's responses:

In [ ]:
def evaluate_response(query: str, tool_used: str, response: str) -> dict:
    """Evaluate an agent's response using the Granite model"""

    prompt = f"""<think>
Please evaluate this AI agent interaction:

User Query: {query}
Tool Used: {tool_used}
Agent Response: {response}

Rate on a scale of 1-5 (5 being best) and provide a brief explanation.
Return your evaluation in this JSON format:
{{"score": <1-5>, "explanation": "<your brief explanation>"}}
</think>"""

    try:
        # Get evaluation from Granite
        result = model.predict(prompt, max_new_tokens=200)
        return json.loads(result.strip())
    except:
        return {"score": 0, "explanation": "Evaluation failed"}

# Test the evaluation
test_query = "What's 5 plus 3?"
test_response = agent.use_calculator("5 + 3")
eval_result = evaluate_response(test_query, "calculator", test_response)

print(f"Query: {test_query}")
print(f"Response: {test_response}")
print(f"Evaluation Score: {eval_result['score']}/5")
print(f"Explanation: {eval_result['explanation']}")

## 4. Try It Yourself!

Now you can try different queries and evaluate the responses. Here's an example to get you started:

In [ ]:
def try_agent(query: str, tool: str, input_value: str):
    """Test the agent with different queries"""
    # Get the right tool function
    if tool == "calculator":
        response = agent.use_calculator(input_value)
    elif tool == "weather":
        response = agent.check_weather(input_value)
    elif tool == "text":
        response = agent.analyze_text(input_value)
    else:
        return "Unknown tool"

    # Evaluate the response
    evaluation = evaluate_response(query, tool, response)

    # Print results
    print(f"Query: {query}")
    print(f"Tool Used: {tool}")
    print(f"Response: {response}")
    print(f"Evaluation Score: {evaluation['score']}/5")
    print(f"Feedback: {evaluation['explanation']}")

# Example usage:
print("Example 1: Calculator")
try_agent("What's 10 times 5?", "calculator", "10 * 5")

print("\nExample 2: Weather")
try_agent("What's the weather in Tokyo?", "weather", "Tokyo")

print("\nExample 3: Text Analysis")
try_agent("How does this text sound?", "text", "This is a great and wonderful day!")

## Your Turn!

Try creating your own queries below. Here are some ideas:
- Try different calculations
- Check weather for different cities
- Analyze different text samples

Just copy and modify the example below:

In [ ]:
# Your tests here!
try_agent(
    query="What's 25 divided by 5?",  # Your question
    tool="calculator",                # Choose: calculator, weather, or text
    input_value="25 / 5"             # Your input
)

## 🎉 Congratulations!

You've completed the simple AI agent evaluation lab! You've learned:
- How to work with a simple AI agent
- How to use different tools
- How to evaluate AI responses

Feel free to experiment with different queries and tools!